## Loopio API

Documentation of the different API endpoints used for Loopio
- https://developer.loopio.com/docs/loopio-api/55ef47e33f5eb-list-library-entries-you-can-interact-with
    - Argument to filter as part of the query parameters 
    - Is it possible to filter based on tags are part of the API query 
- https://developer.loopio.com/docs/loopio-api/6e8ee29b39bf0-update-core-properties-of-a-library-entry
    - Endpoint to update the properties of a library entry, will use to tag 
- https://developer.loopio.com/docs/loopio-api/bedaddd3877a9-create-a-library-entry
    - Endpoint to create a new library entry 


## Interacting with the Loopio API 
1. Use API tokens to checkout a temporary bearer token 
2. Use the Loopio API to read the library entries

In [ ]:
import requests
import json

### Function to get the Bearer Authorization Token
----

`get_secrets` helper function retrieves `id` and `secret` from a file  
`get_auth_token` queries the auth token route and returns the bearer token for later use

In [ ]:
def get_secrets(fn,secret): 
    """Small helper to get secrets from the file"""
    f = open(fn)
    dict = json.load(f)
    return dict[secret]
    

In [ ]:
def get_auth_token(secrets_file:str,scope:str) -> str: 
    """Function to get the oauth2 bearer token from the Loopio API
        Inputs: Secrets File Location (str)
        Outputs: Bearer Token (str)
    """
    host = "https://api.int01.loopio.com/oauth2/access_token"
    id = get_secrets(secrets_file,"id")
    secret = get_secrets(secrets_file,"secret")
    headers = {
        "content_type": "application/x-www-form-urlencoded"
    }   
    payload = { 
        "grant_type":"client_credentials", 
        "client_id":id,
        "client_secret": secret,
        "scope":scope
    }
    r = requests.post(host,headers=headers, data=payload)
    return r.json()["access_token"]

bearer_token = get_auth_token("./secrets/loopio_secret", scope="library:read")

### API Integration #1
    Get a list of Library Entries
----

**Purpose:** Build an inventory of all the library entries associated with a project  
**Documentation:** https://loopio.stoplight.io/docs/loopio-api/55ef47e33f5eb-list-library-entries-you-can-interact-with

In [ ]:
def get_library_entries():
    """Function to get the library entries available in this Loopio Instance"""
    get_auth_token(secrets_file="./secrets/loopio_secret", scope="library:read")

    url = "https://api.int01.loopio.com/data/v2/libraryEntries"
    headers = { 
        "Accept":"application/json",
        "Authorization": "Bearer " + bearer_token
    }
    r = requests.get(url,headers=headers)
    return r.json()

entries = get_library_entries()

### API Integration #2
    Update a library Entry Properties
----

**Purpose:** Enable the editing of a specified file. This would be used to flag a library entry as translated or not  
**Documentation:** https://loopio.stoplight.io/docs/loopio-api/6e8ee29b39bf0-update-core-properties-of-a-library-entry

In [225]:
first_entry = entries["items"][0]
id = first_entry["id"]
bearer_token = get_auth_token(secrets_file="./secrets/loopio_secret",scope="library:write")
id


565933

In [221]:
url = "https://api.int01.loopio.com/data/v2/libraryEntries/{id}"
url = url.format(id=id)
headers = {
    "Accept":"application/json",
    "Authorization": "Bearer " + bearer_token,
    "Content-Type": "application/json-patch+json",
}
data = [
    {
    "path":"/languageCode",
    "op": "replace",
    "value": "en"
    }, 
    {"path":"/tags/0", "op":"add", "value":"new-val"}
]

r = requests.patch(url.format(libraryEntryId=id), headers=headers, data=json.dumps(data))
r.json()

In [224]:
def patch_update_library_entry(id, json_patch:list):
    """Update a library entry
        Inputs: Object ID
        json_patch: Array of JSON objects representing the patch

        Example to change the Language Code to Spanish: 
        data = [
            {"path":"/languageCode", "op":"replace", "value":"es"}
        ]
    """
    # Get the Bearer Token 
    bearer_token = get_auth_token(secrets_file="./secrets/loopio_secret",scope="library:write")

    # Construct the Request
    url = "https://api.int01.loopio.com/data/v2/libraryEntries/{id}"
    url = url.format(id=id)
    headers = {
        "Accept":"application/json",
        "Authorization": "Bearer " + bearer_token,
        "Content-Type": "application/json-patch+json",
    }

    # Execute the Patch Request 
    r = requests.patch(url.format(libraryEntryId=id), headers=headers, data=json.dumps(json_patch))
    return r.status_code

In [ ]:
json_patch = 
patch_update_library_entry(565933)